In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import shap
import matplotlib.pyplot as plt

In [ ]:
# Constants for labels
Bankruptcy = 'Bankruptcy'
Non_Bankruptcy = 'Non_Bankruptcy'

In [ ]:
# Function to load images from a folder and extract company names
def load_images_from_folder(folder, label):
    images = []
    company_names = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder, filename)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
            image_array = tf.keras.preprocessing.image.img_to_array(image)
            images.append(image_array)
            company_names.append(filename.split('.')[0])
    labels = [label] * len(images)
    return images, labels, company_names

In [ ]:
# Load Bankruptcy and Non-Bankruptcy images
bankruptcy_images, bankruptcy_labels, bankruptcy_companies = load_images_from_folder(r'C:\Users\Administrator\CNN Code - Github\Data\Bankruptcy', Bankruptcy)
non_bankruptcy_images, non_bankruptcy_labels, non_bankruptcy_companies = load_images_from_folder(r'C:\Users\Administrator\CNN Code - Github\Data\Non_Bankruptcy', Non_Bankruptcy)

In [ ]:
# Combine and prepare data
images = np.array(bankruptcy_images + non_bankruptcy_images)
labels = np.array(bankruptcy_labels + non_bankruptcy_labels)
company_names = bankruptcy_companies + non_bankruptcy_companies

In [ ]:
# Split the data
X_train, X_test, y_train, y_test, company_names_train, company_names_test = train_test_split(
    images, labels, company_names, test_size=0.2, stratify=labels, random_state=42
)

In [ ]:
# Normalize the image data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Load pre-trained model
model = tf.keras.models.load_model('CNNFS.h5')

In [ ]:
# Get raw predictions
predictions = model.predict(X_test[0:20])

In [ ]:
print("Predicted Image:", predictions)

In [ ]:
# Define a masker
masker = shap.maskers.Image("inpaint_telea", X_train[0].shape)

In [ ]:
# Create an explainer with the model prediction function, the image masker, and class names as strings
explainer = shap.Explainer(model.predict, masker, output_names=[Bankruptcy, Non_Bankruptcy])

In [ ]:
# Explain a subset of images
shap_values = explainer (X_test[0:200], max_evals=2000, batch_size=50, outputs=shap.Explanation.argsort.flip[:2])


In [ ]:
# Custom function to plot images with SHAP values and company names
def plot_with_labels(images, shap_values, company_names, num_images=20):
    num_images = min(num_images, len(images))
    for i in range(num_images):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(images[i] / 255.0)  # Assuming images are not normalized
        plt.title(f"Company: {company_names[i]}")
        plt.axis('off')
        plt.subplot(1, 2, 2)
        shap.image_plot(shap_values[i:i+1], show=False)
        plt.show()

In [ ]:
# Visualize the SHAP values with company names
plot_with_labels(X_test[0:20], shap_values, company_names_test[0:20])